Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train basic model on the generated emnist-lines dataset

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
# To be able to write code as if this notebook was one level up in files tree structure.
sys.path.append('..')

from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops import ctc_ops
from tensorflow.python.ops import math_ops as tf_math_ops

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Input, Lambda, Reshape, TimeDistributed, LSTM
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence

import pandas as pd
from PIL import Image

In [4]:
from recognizer.datasets import Dataset
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet
from recognizer.networks import simple
from recognizer.networks import NetworkInput

Hyperparameters

In [5]:
batch_size = 256
epochs = 4  # 32

In [6]:
model_save_path = Path("../recognizer/weights/lines_time_distributed_model.h5")

## Load characters mapping

In [7]:
import json

with open(Dataset.raw_data_path()/"emnist"/"emnist_essentials.json") as json_file:  
    mapping = json.load(json_file)["mapping"]
    mapping = {m[0]: m[1] for m in mapping}
    mapping[62] = ' '
    mapping[63] = '_'
    
mapping_reversed = {v: k for k, v in mapping.items()}

## Build the new model

In [8]:
char_image_height = 28
char_image_width = 28
max_length = 34
num_classes = 64
image_height = char_image_height
image_width = char_image_width * max_length

input_shape = (image_height, image_width)
output_shape = (max_length, num_classes)

window_width: float = 16
window_stride: float = 8
    
num_windows = ((image_width - window_width) // window_stride) + 1
if num_windows < max_length:
    raise ValueError(f'Window width/stride need to generate >= {max_length} windows (currently {num_windows})')

In [9]:
def slide_window(image, window_width, window_stride):
    """
    Takes (image_height, image_width, 1) input,
    Returns (num_windows, image_height, window_width, 1) output, where
    num_windows is floor((image_width - window_width) / window_stride) + 1
    """
    patches = tf.image.extract_image_patches(image, 
                                             sizes=[1, 1, window_width, 1], 
                                             strides=[1, 1, window_stride, 1], 
                                             rates=[1, 1, 1, 1], 
                                             padding='VALID')
    patches = tf.transpose(patches, (0, 2, 1, 3))
    patches = tf.expand_dims(patches, -1)
    
    return patches

In [10]:
window_width = 12
window_stride = 5

image_input = Input(shape=input_shape, name='image')
image_reshaped = Reshape((image_height, image_width, 1))(image_input)

image_patches = Lambda(
    slide_window,
    arguments={'window_width': window_width, 'window_stride': window_stride}
)(image_reshaped)

In [11]:
image_patches

<tf.Tensor 'lambda/ExpandDims:0' shape=(None, 189, 28, 12, 1) dtype=float32>

### The model

In [12]:
def ctc_decode(y_pred, input_length, max_output_length):
    """
    Cut down from https://github.com/keras-team/keras/blob/master/keras/backend/tensorflow_backend.py#L4170

    Decodes the output of a softmax.
    Uses greedy (best path) search.

    # Arguments
        y_pred: tensor `(samples, time_steps, num_categories)`
            containing the prediction, or output of the softmax.
        input_length: tensor `(samples, )` containing the sequence length for
            each batch item in `y_pred`.
        max_output_length: int giving the max output sequence length

    # Returns
        List: list of one element that contains the decoded sequence.
    """
    y_pred = tf_math_ops.log(tf.transpose(y_pred, perm=[1, 0, 2]) + K.epsilon())
    input_length = tf.cast(tf.squeeze(input_length, axis=-1), tf.int32)

    (decoded, _) = ctc_ops.ctc_greedy_decoder(inputs=y_pred, sequence_length=input_length)

    sparse = decoded[0]
#     decoded_dense = tf.sparse.to_dense(sparse.indices, sparse.dense_shape, sparse.values, default_value=-1)
    decoded_dense = tf.sparse.to_dense(sparse, default_value=-1)

    # Unfortunately, decoded_dense will be of different number of columns, depending on the decodings.
    # We need to get it all in one standard shape, so let's pad if necessary.
    max_length = max_output_length + 2  # giving 2 extra characters for CTC leeway
    cols = tf.shape(decoded_dense)[-1]

    def pad():
        return tf.pad(decoded_dense, [[0, 0], [0, max_length - cols]], constant_values=-1)

    def noop():
        return decoded_dense

    return tf.cond(tf.less(cols, max_length), pad, noop)

In [13]:
convnet = lenet(NetworkInput(input_shape=(image_height, window_width, 1), number_of_classes=num_classes))
convnet.layers

In [14]:
# Get rid of the last two layers (dropout and softmax)
convnet = KerasModel(inputs=convnet.inputs, outputs=convnet.layers[-2].output)
time_distributed_outputs = TimeDistributed(convnet)(image_patches)

lstm_outputs = LSTM(128, return_sequences=True)(time_distributed_outputs)

softmax_output = Dense(num_classes, activation='softmax', name='softmax_output')(lstm_outputs)

y_true = Input(shape=(max_length,), name='y_true')
input_length = Input(shape=(1,), name='input_length')
label_length = Input(shape=(1,), name='label_length')

input_length_processed = Lambda(
    lambda x, num_windows=None: x * num_windows,
    arguments={'num_windows': num_windows}
)(input_length)

ctc_loss_output = Lambda(
    lambda x: K.ctc_batch_cost(x[0], x[1], x[2], x[3]),
    name='ctc_loss'
)([y_true, softmax_output, input_length_processed, label_length])

ctc_decoded_output = Lambda(
    lambda x: ctc_decode(x[0], x[1], max_length),
    name='ctc_decoded'
)([softmax_output, input_length_processed])
    
model = KerasModel(
        inputs=[image_input, y_true, input_length, label_length],
        outputs=[ctc_loss_output, ctc_decoded_output]
    )

W0802 02:10:54.277757 140463248213376 deprecation.py:323] From /opt/anaconda3/envs/line-reader/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5151: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0802 02:10:54.376111 140463248213376 deprecation.py:323] From /opt/anaconda3/envs/line-reader/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5130: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [15]:
softmax_output

<tf.Tensor 'softmax_output/truediv:0' shape=(None, 189, 64) dtype=float32>

In [16]:
input_length_processed

<tf.Tensor 'lambda_1/mul:0' shape=(None, 1) dtype=float32>

In [17]:
ctc_loss_output

<tf.Tensor 'ctc_loss/ExpandDims:0' shape=(None, 1) dtype=float32>

In [18]:
ctc_decoded_output

<tf.Tensor 'ctc_decoded/cond/Identity:0' shape=(None, None) dtype=int64>

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 28, 952)]    0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 28, 952, 1)   0           image[0][0]                      
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 189, 28, 12,  0           reshape[0][0]                    
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 189, 128)     412160      lambda[0][0]                     
____________________________________________________________________________________________

Load emnist lines and pass it to the new model

## Look at the data

In [20]:
df = pd.read_csv(Dataset.processed_data_path()/"emnist_lines"/"data.csv")

output_length = len(df)

In [21]:
df.head()

,image,sentence
0,0.png,Whats wrong at state _____________
1,1.png,The transducer itself moves the __
2,2.png,There is an ancient and venerable
3,3.png,They destroyed a trading house ___
4,4.png,Alec waited a moment on guard ___


In [22]:
image = Image.open(Dataset.processed_data_path()/"emnist_lines"/"0.png")
image

In [23]:
np.array(image).shape

(28, 952)

## Predict on one image before training

In [24]:
[print(layer.name) for layer in model.layers]

image
reshape
lambda
time_distributed
unified_lstm
input_length
y_true
softmax_output
lambda_1
label_length
ctc_loss
ctc_decoded


[None, None, None, None, None, None, None, None, None, None, None, None]

In [25]:
softmax_output.shape[1]

189

In [26]:
input_length

<tf.Tensor 'input_length:0' shape=(None, 1) dtype=float32>

In [27]:
convnet = KerasModel(inputs=model.inputs, outputs=model.layers[-5].output)

image = Image.open(Dataset.processed_data_path()/"emnist_lines"/"0.png")
image_numpy = np.array(image).astype(np.float32) / 255

# Get the prediction and confidence using softmax_output_fn, passing the right input into it.
input_image = np.expand_dims(image_numpy, 0)
softmax_output = convnet(input_image)

input_length = np.array([softmax_output.shape[1]])
decoded, log_prob = K.ctc_decode(softmax_output, input_length, greedy=True)

pred_raw = K.eval(decoded[0])[0]
pred = ''.join(mapping[label] for label in pred_raw).strip()

neg_sum_logit = K.eval(log_prob)[0][0]
conf = np.exp(-neg_sum_logit)

pred, conf

W0802 02:10:55.419102 140463248213376 deprecation.py:323] From /opt/anaconda3/envs/line-reader/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5207: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


('fUfdUdfdfUfjdjdJUf2jdUfdfdfdGq1L101L', 0.0)

## Train it

In [28]:
def format_batch_ctc(batch_x, batch_y):
    """
    Because CTC loss needs to be computed inside of the network, we include information about outputs in the inputs.
    """
    batch_size = batch_y.shape[0]
    y_true = np.argmax(batch_y, axis=-1)
    
    label_lengths = []
    for ind in range(batch_size):
        # Find all of the indices in the label that are blank
        empty_at = np.where(batch_y[ind, :, -1] == 1)[0]
        # Length of the label is the pos of the first blank, or the max length
        if empty_at.shape[0] > 0:
            label_lengths.append(empty_at[0])
        else:
            label_lengths.append(batch_y.shape[1])

    batch_inputs = {
        'image': batch_x,
        'y_true': y_true,
        # TODO :: Remove the lambda layer form the model and change the size here
        'input_length': np.ones((batch_size, 1)),  # dummy, will be set to num_windows in network
        'label_length': np.array(label_lengths)
    }
    batch_outputs = {
        'ctc_loss': np.zeros(batch_size),  # dummy
        'ctc_decoded': y_true
    }
    return batch_inputs, batch_outputs

In [29]:
class LinesDataSequence(Sequence):
    
    def __init__(self, batch_size=32, augment_fn=None, format_fn=None):
        self.df = pd.read_csv(Dataset.processed_data_path()/"emnist_lines"/"data.csv")
        # Shuffle data and reset their index.
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        
        self.batch_size = batch_size
        self.augment_fn = augment_fn
        self.format_fn = format_fn

    def __len__(self):
        return len(df) // batch_size

    def __getitem__(self, idx):
        """Return a single batch."""
        begin = idx * self.batch_size
        end = (idx + 1) * self.batch_size

        batch_x = []
        batch_y = []
        for index in range(begin, end):
            row = self.df.iloc[index]
            
            image = Image.open(Dataset.processed_data_path()/"emnist_lines"/row['image'])
            x = np.array(image).astype(np.float32).reshape(image_height, image_width)
            batch_x.append(x)

            y = [mapping_reversed[char] for char in row['sentence']]
            y = to_categorical(y, num_classes).astype(np.int)
            batch_y.append(y)
            
        batch_x, batch_y = np.array(batch_x), np.array(batch_y)

        return format_batch_ctc(batch_x, batch_y)

In [30]:
def loss():
    """Dummy loss function: just pass through the loss that we computed in the network."""
    return {'ctc_loss': lambda y_true, y_pred: y_pred}

Check progress in TensorBoard by running `tensorboard --logdir=logs/lines_time_distributed_ctc` in terminal

In [1]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='../logs/lines_time_distributed_ctc')
]

model.compile(loss=loss(), optimizer=RMSprop(), metrics=['accuracy'])
model.fit_generator(LinesDataSequence(batch_size),
                    steps_per_epoch=len(df) // batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    use_multiprocessing=True,
                    workers=2)
#                     shuffle=True)
#                     validation_data=)

NameError: name 'keras' is not defined

# Save the model

In [ ]:
model.save(model_save_path)

# keras.models.save_model(
#     model,
#     model_save_path,
#     overwrite=True,
#     include_optimizer=True
# )

## Restore the model

In [ ]:
# model = keras.models.load_model(model_save_path)

## Evaluate

In [35]:
convnet = KerasModel(inputs=model.inputs, outputs=model.layers[-5].output)

image = Image.open(Dataset.processed_data_path()/"emnist_lines"/"0.png")
image_numpy = np.array(image).astype(np.float32) / 255

# Get the prediction and confidence using softmax_output_fn, passing the right input into it.
input_image = np.expand_dims(image_numpy, 0)
softmax_output = convnet(input_image)

input_length = np.array([softmax_output.shape[1]])
decoded, log_prob = K.ctc_decode(softmax_output, input_length, greedy=True)

pred_raw = K.eval(decoded[0])[0]
pred = ''.join(mapping[label] for label in pred_raw).strip()

neg_sum_logit = K.eval(log_prob)[0][0]
conf = np.exp(-neg_sum_logit)

pred, conf

('', 0.019819042)